   ### Understanding the Action Pipeline

In a system utilizing **Retrieval-Augmented Generation (RAG)**, the action pipeline goes far beyond simple information retrieval. After data is transformed into embeddings and organized within a **vector store**, this stage defines how the agent processes the retrieved information and decides which action to execute to construct a contextualized response. This approach not only ensures the integration between raw data and elaborated answers but also allows the system to apply specific business rules and logic to refine the final output.



### Structure and Operational Flow

The action pipeline is composed of several modules acting in sequence. Generally, following the semantic query in the vector store:

1.  The agent collects the most relevant snippets based on **semantic similarity**.
2.  **Validations and transformations** are applied to ensure the data is coherent with the requested context.
3.  The system can then trigger **custom functions** or execute additional checks before forming the final response for the user.

An illustrative example in pseudocode:

```python
# Data Retrieval
snippets = vector_store.retrieve(query, k=3)

# Snippet processing and verification
validated_data = []
for snippet in snippets:
    if validate_snippet(snippet):
        validated_data.append(transform_snippet(snippet))

# Response construction
response = generator_model.generate_response(context=validated_data, question=query)
```

   This modular flow allows each stage to be adjusted according to the domain's complexity and the specific needs of the application, increasing the conversational agent's reliability.

### Critical Aspects and Key Considerations



Among the main advantages of this approach, the following stand out:

* **Flexibility:** The ability to intersperse validation, transformation, and verification functions allows the system to adapt to different scenarios and domain-specific rules—for example, interpreting complex football rules in an explainer agent.
* **Response Precision:** By integrating an intermediate layer of actions, the system can filter out irrelevant information and ensure the final response is aligned with both the original data and the defined business logic.



On the other hand, it is important to consider that a more complex action pipeline may require greater development and testing efforts, as well as special care regarding the **orchestration** between modules to avoid delays or integration errors.

This approach highlights how a well-structured flow, which is not limited solely to data retrieval, can significantly contribute to the quality and relevance of responses generated by a RAG-based conversational agent.

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import Chroma
from langchain_classic.chains import RetrievalQA

C:\Users\ana\Documents\Cursos\aluraDataScience\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 1. Problem definition

In [2]:
CAMINHO_PDF = "../data/Regras_do_Jogo_2023_24.pdf"
loader = PyPDFLoader(CAMINHO_PDF)
docs = loader.load()

len(docs)

232

#### 2. Docs preparation

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(docs)

len(chunks)

654

#### 3. Embeddings and vectorstore

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="chroma_regras_futebol"
)

#### 4. Retriever

In [5]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

#### 5. LLM integration

In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash"
)